# Solver for Qodec's mystery products sudoku
https://logic-masters.de/Raetselportal/Raetsel/zeigen.php?id=00047R

In [ ]:
from ortools.sat.python import cp_model

## Define the Sudoku board

In [2]:
model = cp_model.CpModel()
def create_row_vars(y):
    return [model.NewIntVar(1, 9, 'x:{},y:{}'.format(x,y)) for x in range(9)] 
nodes = [create_row_vars(y) for y in range(9)]

## Add the normal sudoku constraints

In [3]:
for x in range(9):
    model.AddAllDifferent(nodes[x])

In [4]:
for y in range(9):
    model.AddAllDifferent([nodes[x][y] for x in range(9)])

In [5]:
for x_offset in range(0, 9, 3):
    for y_offset in range(0, 9, 3):
        square = []
        for x in range(3):
            for y in range(3):
                square.append(nodes[x_offset+x][y_offset+y])
        model.AddAllDifferent(square)

## Defining cage groups

In [6]:
import math
def cage_group(name, cg, model):
    lengths = [len(cage) for cage in cg]
    min_len = min(lengths)
    max_len = max(lengths)
    min_magic = math.factorial(max_len)
    max_magic = int(math.factorial(9)/math.factorial(9-min_len))
    magic_nr = model.NewIntVar(min_magic, max_magic, 'magic_nr_{}'.format(name))
    for i, cage in enumerate(cg):
        model.AddAllDifferent(cage)
        if len(cage)==1:
            model.Add(magic_nr==cage[0])
        elif len(cage)==2:
            model.AddMultiplicationEquality(magic_nr,[cage[0],cage[1]])
        else:
            mul = model.NewIntVar(1, max_magic, 'magic_mul{}{}_{}'.format(i,1,name))
            model.AddMultiplicationEquality(mul, [cage[0], cage[1]])         
            for j in range(2,len(cage)-1):
                prev_mul = mul
                mul = model.NewIntVar(1, 9**min_len, 'magic_mul{}{}_{}'.format(i,j,name))
                model.AddMultiplicationEquality(mul, [prev_mul, cage[j]])                        
            model.AddMultiplicationEquality(magic_nr,[mul,cage[len(cage)-1]])
    return magic_nr

## Defining the puzzle

In [7]:
model.Add(nodes[0][0]==7)

In [8]:
cage_group("blue",[[nodes[2][1],nodes[1][2],nodes[2][2]],
                   [nodes[4][3],nodes[3][4],nodes[4][4]],
                   [nodes[6][5],nodes[6][6]],
                   [nodes[4][8],nodes[5][8]]
                  ], model)

magic_nr_blue(6..72)

In [9]:
cage_group("green",[[nodes[3][0],nodes[4][0],nodes[3][1]],
                    [nodes[8][4],nodes[8][5]],
                    [nodes[6][7], nodes[7][7], nodes[6][8], nodes[7][8]]
                   ], model)

magic_nr_green(24..72)

In [10]:
cage_group("yellow",[[nodes[7][0],nodes[8][0]],
                     [nodes[0][4]],
                     [nodes[0][7],nodes[0][8]]
                   ], model)

magic_nr_yellow(2..9)

In [11]:
cage_group("purple",[[nodes[5][2],nodes[5][3],nodes[6][3]],
                     [nodes[2][5],nodes[3][5],nodes[3][6]],
                     [nodes[5][6],nodes[4][7],nodes[5][7]],
                     [nodes[7][4],nodes[7][5],nodes[7][6],nodes[8][6]],
                     [nodes[8][7],nodes[8][8]]
                   ], model)

magic_nr_purple(24..72)

## Solving the puzzle

In [12]:
solver = cp_model.CpSolver()
solver.Solve(model)

4

In [13]:
def print_solution(solver, nodes):
    for y in range(9):
        if(y%3==0):
            print("")            
        for x in range(9):
            if(x%3 == 0):
                print(" ", end='')
            print(solver.Value(nodes[x][y]), end='')            
        print("")
    

In [14]:
print_solution(solver,nodes)


 794 325 681
 628 791 453
 315 864 792

 137 689 245
 842 517 936
 569 432 817

 981 273 564
 253 946 178
 476 158 329


In [15]:
print(solver.ResponseStats())

CpSolverResponse:
status: OPTIMAL
objective: 0
best_bound: 0
booleans: 215
conflicts: 10769662
branches: 15102383
propagations: 232799944
integer_propagations: 776360156
walltime: 1104.53
usertime: 1104.53
deterministic_time: 345.749
primal_integral: 0

